In [1]:
import datasets
from collections import defaultdict
import tqdm
import random
import json

In [6]:
PREFERRED_PROMPT = """You are an expert in {subject}. Answer the following multiple choice question. The first character in the response should be the selected multiple choice letter. Do NOT ouptut the correct answer at the end but instead at the beginning.The following multiple choice question has the correct answer {correct_ans}. First output the correct answer then provide an explanation of why {correct_ans} is the correct answer. \n{question}"""
DISPREFERRED_PROMPT = "Answer the following multiple choice question. The first character in the response should be the selected multiple choice letter. Do NOT ouptut the correct answer at the end but instead at the beginning. Question: {question}"

In [18]:
prompts = []

with open("/iris/u/sherylh/multiple-teacher-generalization/data_gen/together/easy_test_fullmmlu.json", "r") as f:
    json_obj = json.load(f)
    for ind, obj in enumerate(json_obj):
        prompts.append({
            **obj,
            "preferred_prompt": PREFERRED_PROMPT.format(subject=obj["subject"], correct_ans=obj["correct_ans"], question=obj["prompt"]),
            "dispreferred_prompt": DISPREFERRED_PROMPT.format(question=obj["prompt"])
        })
print("loaded", len(prompts), "examples")

loaded 862 examples


In [19]:
import sys
sys.path.insert(1, 'api')
from togethermodel import TogetherModel


llama = TogetherModel(['meta-llama/Llama-3-70b-chat-hf'])
qwen = TogetherModel(['Qwen/Qwen1.5-1.8B-Chat'])

res_p = llama.get_responses([i["preferred_prompt"] for i in prompts])
res_d = qwen.get_responses([i["dispreferred_prompt"] for i in prompts])


('meta-llama/Llama-3-70b-chat-hf', 'You are an expert in other. Answer the following multiple choice question. The first character in the response should be the selected multiple choice letter. Do NOT ouptut the correct answer at the end but instead at the beginning.The following multiple choice question has the correct answer A. First output the correct answer then provide an explanation of why A is the correct answer. \nHow is 4:00 pm expressed in military time?\nA. 1600\nB. 4\nC. 400\nD. 4:00', '303e07102419a98aae9be912b8e612cf16c0a54babfde4927bf4e86e9a4b76d6', 0.7, 1, 0.9)


100%|██████████| 862/862 [02:31<00:00,  5.67it/s]


('Qwen/Qwen1.5-1.8B-Chat', 'Answer the following multiple choice question. The first character in the response should be the selected multiple choice letter. Do NOT ouptut the correct answer at the end but instead at the beginning. Question: How is 4:00 pm expressed in military time?\nA. 1600\nB. 4\nC. 400\nD. 4:00', '303e07102419a98aae9be912b8e612cf16c0a54babfde4927bf4e86e9a4b76d6', 0.7, 1, 0.9)


100%|██████████| 862/862 [02:30<00:00,  5.74it/s]


In [21]:
out = []
for obj, p, d in zip(prompts, res_p, res_d):
    obj["preferred_response"] = p['output']
    obj["dispreferred_response"] = d['output']
    obj["preferred_correct"] = obj["preferred_response"].strip()[0].lower() == obj["correct_ans"].lower()
    obj["dispreferred_correct"] = obj["dispreferred_response"].strip()[0].lower() == obj["correct_ans"].lower()

    out.append(obj)
with open("/iris/u/sherylh/multiple-teacher-generalization/data_gen/preference_dataset/mmlu_easy_test.json", "w+") as f:
    json.dump(out, f)